# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
# Your code here:
employee = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
employee.shape

(33183, 8)

In [4]:
# Your code here:
employee.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
# Your code here:
employee.isna().sum()



Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
# Your code here:
employee['Salary or Hourly'].value_counts()


Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [7]:
# Your code here:
employee.groupby("Department").agg({"Name":"count"}).reset_index().sort_values(by='Name', ascending= False)

,Department,Name
27,POLICE,13414
17,FIRE,4641
31,STREETS & SAN,2198
26,OEMC,2102
34,WATER MGMNT,1879
2,AVIATION,1629
32,TRANSPORTN,1140
30,PUBLIC LIBRARY,1015
18,GENERAL SERVICES,980
15,FAMILY & SUPPORT,615


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [8]:
# Your code here:
import scipy.stats as st

In [9]:
# 1. Set the hypothesis

# H0: mu != 30
# H1: mu == 30
n = 30

In [10]:
# 2. Choose significance / confidence level
a = 0.05

In [11]:
# 3. Sample
employee_hour = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate']
employee_hour

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [12]:
# 4. Compute statistic
# 5. Get p-value

statistic, p_value =st.ttest_1samp(employee_hour, n)

print(statistic)
print(p_value)

p_value < a

20.6198057854942
4.3230240486229894e-92


True

In [13]:
employee_hour.mean()

32.78855771628128

In [14]:
# print the results
print("t-statistic:", statistic)
print("p-value:", p_value)

t-statistic: 20.6198057854942
p-value: 4.3230240486229894e-92


In [15]:
# check if the p-value is less than the significance level (alpha)
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis")
else:
    print("Fail to reject the null hypothesis")

Reject the null hypothesis


In [16]:
# 6. Decide
# if p value is lower that the significance level we reject the null 
# if p value is equal or bigger that the signiface level we do not reject 

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [17]:
t1 = employee[employee['Salary or Hourly'] == 'Salary']['Annual Salary']
t1.mean()

86786.99979015153

In [18]:
# set the null hypothesis value
sample_police=employee[(employee["Salary or Hourly"]=="Salary") & (employee["Department"]=="POLICE")].sample(200)

#Setting the hypothesis:
#h0 mu>=86000
#h1mu<86000

st.ttest_1samp(sample_police["Annual Salary"], 86000, alternative="greater")
st.ttest_1samp(sample_police["Annual Salary"], 86000, alternative="less")

Ttest_1sampResult(statistic=-0.11181480480142572, pvalue=0.45554147517134447)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [19]:
employee.head(2)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN


In [20]:
# Your code here:
hourly_workers=employee[employee["Salary or Hourly"]=="Hourly"]

pd.crosstab(index=hourly_workers['Department'], columns=hourly_workers["Salary or Hourly"]).reset_index().sort_values(by='Hourly', ascending=False)


Salary or Hourly,Department,Hourly
19,STREETS & SAN,1862
21,WATER MGMNT,1513
15,OEMC,1273
1,AVIATION,1082
10,GENERAL SERVICES,765
20,TRANSPORTN,725
18,PUBLIC LIBRARY,299
7,FAMILY & SUPPORT,287
4,CITY COUNCIL,64
8,FINANCE,44


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [ ]:
# Your code here:
sample_d=employee[(employee["Salary or Hourly"]=="Hourly") & (employee["Department"]=="STREETS & SAN")].sample(200)

#Setting the hypothesis:
#h0 mu<=35
#h1mu>35

st.ttest_1samp(sample_d["Hourly Rate"], 35, alternative="greater")

Ttest_1sampResult(statistic=-2.5525828627375673, pvalue=0.9942786561333542)

In [ ]:
employee[(employee["Salary or Hourly"]=="Hourly") & (employee["Department"]=="STREETS & SAN")]["Hourly Rate"].mean()

33.728378088076845

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [ ]:
employee.columns

Index(['Name', 'Job Titles', 'Department', 'Full or Part-Time',
       'Salary or Hourly', 'Typical Hours', 'Annual Salary', 'Hourly Rate'],
      dtype='object')

In [22]:
a = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate']
mean = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate'].mean()
std = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate'].std()
n = employee[employee['Salary or Hourly'] == 'Hourly']['Hourly Rate'].count()
a

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [ ]:
st.norm.interval(0.95,loc=mean,scale=std/np.sqrt(n))

(32.52349834813896, 33.05361708442361)

In [ ]:
# Your code here:
st.sem(a, axis=0, ddof=1)

0.1352368565101596

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [ ]:
employee.head(2)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN


In [ ]:
# Your code here:
a = employee[employee['Department'] == 'POLICE']['Annual Salary']
mean = employee[employee['Department'] == 'POLICE']['Annual Salary'].mean()
std = employee[employee['Department'] == 'POLICE']['Annual Salary'].std()
n = employee[employee['Department'] == 'POLICE']['Annual Salary'].count()

display(a)

0        101442.0
1         94122.0
9         93354.0
11        84054.0
12        87006.0
           ...   
33177     72510.0
33178     72510.0
33179     48078.0
33180     90024.0
33181     93354.0
Name: Annual Salary, Length: 13414, dtype: float64

In [ ]:
st.norm.interval(0.95,loc=mean,scale=std/np.sqrt(n))

(86177.08425202279, 86795.74475424399)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

